In [323]:
import pandas as pd
import os
# Read data into papers
papers = pd.read_csv('../Experiments/metadata/songsMetadata.csv')
# Print head
papers.head()

,artistId,geniusName,songShortcut,title,songId,url,lyrics,releaseDate
0,11774,20Syl,20SYL1,100% Autoproduction by Hocus Pocus,160272,https://genius.com/Hocus-pocus-100-autoproduct...,\n \n \n [Scrat...,000000000
1,11774,20Syl,20SYL2,100 Grammes de Peur by Hocus Pocus,108615,https://genius.com/Hocus-pocus-100-grammes-de-...,\n \n \n On la ...,000000000
2,11774,20Syl,20SYL3,10 que tu penses by Hocus Pocus,2319837,https://genius.com/Hocus-pocus-10-que-tu-pense...,\n \n \n [Intro...,"January 1, 1998"
3,11774,20Syl,20SYL4,10 YRS by End of the Weak France (Ft. Artik (Q...,438039,https://genius.com/End-of-the-weak-france-10-y...,\n \n \n [Artik...,"February 26, 2014"
4,11774,20Syl,20SYL6,73 Touches by Hocus Pocus,61676,https://genius.com/Hocus-pocus-73-touches-lyrics,\n \n \n [Coupl...,000000000


In [325]:
# Remove the columns
papers = papers.drop(columns=['artistId', 'geniusName', 'songShortcut', 
                              'title', 'songId', 'url', 'releaseDate'], axis=1)
                              
# Print out the first rows of papers
papers.head()

,lyrics
0,\n \n \n [Scrat...
1,\n \n \n On la ...
2,\n \n \n [Intro...
3,\n \n \n [Artik...
4,\n \n \n [Coupl...


In [328]:
# Load the regular expression library
import re
# Remove punctuation
# Remove sections seprators 
papers['lyrics'] = papers['lyrics'].map(lambda x: re.sub(r'\[.*\]', '', x))
papers['lyrics'] = papers['lyrics'].map(lambda x: re.sub('couplet', '', x))
papers['lyrics'] = papers['lyrics'].map(lambda x: re.sub('refrain', '', x))
papers['lyrics'] = papers['lyrics'].map(lambda x: re.sub('intro', '', x))
papers['lyrics'] = papers['lyrics'].map(lambda x: re.sub('outro', '', x))
papers['lyrics'] = papers['lyrics'].map(lambda x: re.sub('pré-refrain', '', x))
papers['lyrics'] = papers['lyrics'].map(lambda x: re.sub(r'[,\.!?]', '', x))

# Convert the titles to lowercase
papers['lyrics'] = papers['lyrics'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['lyrics'].head(10)

0    \n          \n            \n            \n"ca ...
1    \n          \n            \n            on la ...
2    \n          \n            \n            \n"i n...
3    \n          \n            \n            \nj'fa...
4    \n          \n            \n            \nje f...
5    \n          \n            \n            \nfive...
6     ×2\nà mi-chemin entre what you think you are\...
7    \nles keufs frappent on n'ouvre pas banlieue s...
8    \néwa rohff \npour tous les massifs authentiqu...
9    (admiral t)\n\nc'est du dancehall que les gens...
Name: lyrics, dtype: object

In [335]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = papers.lyrics.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['ca', 'va', 'faire', 'un', 'sacre', 'boucan', 'autoproduction', 'pour', 'un', 'son', 'pur', 'sang', 'empire', 'sans', 'empereur', 'ca', 'va', 'faire', 'un', 'sacre', 'boucan', 'cigare', 'au', 'bord', 'des', 'levres', 'un', 'verre', 'de', 'whisky']


In [339]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [283]:
# NLTK Stop words
# import nltk
# nltk.download('stopwords')
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english') + stopwords.words('french')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [343]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'fr' model, keeping only tagger component (for efficiency)
nlp = spacy.load('fr_core_news_lg', disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)
print(data_lemmatized[:1])

[['aller', 'sacre_boucan', 'autoproduction', 'pur', 'sang', 'empire', 'empereur', 'aller', 'sacre_boucan', 'cigare', 'bord', 'levre', 'verre', 'whisky', 'bureau', 'secretair', 'dessou', 'repense', 'dernier', 'blaireau', 'entube', 'dire', 'tenir', 'tube', 'recette', 'campagne', 'pub', 'sacem', 'reste', 'aime', 'producteur', 'rap', 'aller', 'croire', 'ca', 'gene', 'contraire', 'contrat', 'enfiler', 'perle', 'ecoute', 'nouveau', 'merde', 'groupe', 'rever', 'percer', 'bon', 'petit', 'truc', 'toucher', 'mal', 'montrer', 'fesse', 'faire', 'jeter', 'tact', 'producteur', 'charmer', 'vente', 'veux', 'seducteur', 'tenir', 'mec', 'venir', 'entrer', 'cassette', 'main', 'pouvoir', 'tube', 'prochain', 'veu', 'gamin', 'vouloir', 'piscine', 'studio', 'sinon', 'cher', 'champagne', 'maillot', 'bain', 'plaisant', 'champagne', 'faire', 'voix', 'girl', 'enfin', 'voir', 'presente', 'tas', 'star', 'vouloir', 'phat', 'genre', 'limousine', 'boulevard', 'nom', 'son', 'radio', 'fans', 'face', 'maillot', 'arrache

In [347]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])
num_of_docs = len(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 6), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 5), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 3), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 2), (54, 3), (55, 3), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 5), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 2), (94, 1), (95, 1), (96, 2), (97, 1), (98, 1), (99, 1), (100, 3), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1)

In [134]:
from gensim.models import CoherenceModel

# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    perplexity = lda_model.log_perplexity(corpus)
    return [coherence_model_lda.get_coherence(), perplexity]

    
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 30
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': [],
                 'Perplexity': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    results = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(results[0])
                    model_results['Perplexity'].append(results[1])
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

[[(0, 1), (1, 1), (2, 1), (3, 6), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 5), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 5), (24, 1), (25, 2), (26, 3), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 2), (56, 3), (57, 3), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 5), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 2), (97, 1), (98, 1), (99, 2), (100, 1), (101, 1), (102, 1), (103, 3), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1)

  0%|          | 0/540 [04:16<?, ?it/s]
1680it [28:02:09, 60.08s/it]


In [351]:
# Final model based on perplexity score

import pickle 

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=29, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.91,
                                           eta=0.01)

import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared                                          

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
24     0.014538  0.022051       1        1  13.888925
11    -0.139820 -0.062608       2        1   6.294458
26     0.167580 -0.006003       3        1   6.140022
0      0.178841  0.032543       4        1   5.397688
23     0.157627  0.010018       5        1   4.791455
5      0.075257  0.103813       6        1   4.053275
6     -0.006906 -0.095013       7        1   3.455798
9     -0.018888  0.025743       8        1   3.332043
18     0.011325  0.033553       9        1   3.252046
27     0.034039 -0.008726      10        1   3.227547
14    -0.021138  0.035220      11        1   2.978435
4      0.100226 -0.046381      12        1   2.932054
20    -0.043832 -0.045002      13        1   2.895221
19    -0.046716  0.074869      14        1   2.804359
12    -0.058753 -0.000738      15        1   2.750902
10     0.040096 -0.053742      16        1   2.707681
17    -0.028972 -0.020196      17        1   2.618932
28    -0.033476 -0.054854      18        1   2.583447
3      0.029660  0.005367      19        1   2.571580
1     -0.039726 -0.011793      20        1   2.356972
8     -0.060598 -0.006222      21        1   2.347660
7     -0.083874 -0.048622      22        1   2.317511
16     0.120061  0.006280      23        1   2.245211
2     -0.017423 -0.035614      24        1   2.237042
13    -0.053965 -0.035214      25        1   2.226532
15    -0.042116 -0.060957      26        1   2.071113
22    -0.040211 -0.008164      27        1   1.943093
21    -0.085036 -0.052082      28        1   1.929351
25    -0.107801  0.302475      29        1   1.649648, topic_info=          Term          Freq         Total Category  logprob  loglift
3        aller  17787.000000  17787.000000  Default  30.0000  30.0000
582       ouai   6475.000000   6475.000000  Default  29.0000  29.0000
261     perdre   3799.000000   3799.000000  Default  28.0000  28.0000
1084     negro   2423.000000   2423.000000  Default  27.0000  27.0000
1         aime   4325.000000   4325.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
774   entendre    147.852245   2050.664427  Topic29  -5.1135   1.4749
1425    jamais    151.018421   4527.298494  Topic29  -5.0923   0.7041
425    prendre    157.177552   8022.739157  Topic29  -5.0524   0.1719
122       plus    158.301639  17135.801019  Topic29  -5.0452  -0.5798
285       seul    137.188515   3749.641285  Topic29  -5.1884   0.7965

[2491 rows x 6 columns], token_table=       Topic      Freq        Term
term                              
10324      1  0.009581  abandonner
10324      3  0.028744  abandonner
10324      5  0.814426  abandonner
10324      6  0.047907  abandonner
10324     18  0.095815  abandonner
...      ...       ...         ...
1165      22  0.003962         œil
1165      23  0.011886         œil
1165      24  0.001981         œil
1165      25  0.049526         œil
1165      28  0.247629         œil

[13577 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[25, 12, 27, 1, 24, 6, 7, 10, 19, 28, 15, 5, 21, 20, 13, 11, 18, 29, 4, 2, 9, 8, 17, 3, 14, 16, 23, 22, 26])

In [355]:
# Final model based on coherence score

import pickle 

lda_model_coherence = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=27, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.91,
                                           eta=0.91)

import pyLDAvis.gensim
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared_coherence = pyLDAvis.gensim.prepare(lda_model_coherence, corpus, id2word)
LDAvis_prepared_coherence                                          

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
24     0.378326 -0.001026       1        1  86.138284
21     0.002308  0.028209       2        1   0.829439
5     -0.010290 -0.008255       3        1   0.688616
14    -0.013387 -0.006730       4        1   0.687655
11    -0.013835 -0.005342       5        1   0.651166
9     -0.012420 -0.004721       6        1   0.642818
19    -0.015611 -0.002053       7        1   0.606764
8     -0.013551 -0.004659       8        1   0.574351
10    -0.016792 -0.000582       9        1   0.552177
7     -0.015517 -0.000795      10        1   0.549132
1     -0.013447 -0.002082      11        1   0.532164
18    -0.015848 -0.000934      12        1   0.531948
15    -0.015773 -0.000799      13        1   0.522686
17    -0.015829  0.000003      14        1   0.496008
2     -0.015967  0.000197      15        1   0.485059
12    -0.016206  0.000289      16        1   0.484913
4     -0.015673  0.000152      17        1   0.482532
3     -0.015832  0.000506      18        1   0.467393
25    -0.016296  0.000714      19        1   0.463725
20    -0.015921  0.000795      20        1   0.458792
22    -0.016244  0.000957      21        1   0.456718
26    -0.015856  0.000925      22        1   0.451603
23    -0.015640  0.000778      23        1   0.451259
16    -0.016307  0.001122      24        1   0.450844
6     -0.016156  0.001100      25        1   0.448966
0     -0.015686  0.000971      26        1   0.447759
13    -0.016550  0.001262      27        1   0.447230, topic_info=           Term         Freq        Total Category  logprob  loglift
1808       bois   633.000000   633.000000  Default  30.0000  30.0000
1084      negro  1742.000000  1742.000000  Default  29.0000  29.0000
1097     parole   559.000000   559.000000  Default  28.0000  28.0000
7524  bouteille   492.000000   492.000000  Default  27.0000  27.0000
206        coup  3463.000000  3463.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
425     prendre     0.185051  6818.247604  Topic27 -10.4916  -5.1046
102      mettre     0.179783  5514.487345  Topic27 -10.5205  -4.9213
633    toujours     0.171469  4277.153434  Topic27 -10.5678  -4.7145
156       temps     0.169908  4515.141169  Topic27 -10.5770  -4.7778
1099     passer     0.169629  5466.830489  Topic27 -10.5786  -4.9707

[1460 rows x 6 columns], token_table=       Topic      Freq                Term
term                                      
49373      1  0.322341           aaah_aaah
27472      1  0.156702  aberration_origine
27472      8  0.313405  aberration_origine
3656       1  0.398153            absorber
3656       4  0.199076            absorber
...      ...       ...                 ...
16283     24  0.187450       zombies_zonme
16284      1  0.181218               zonme
16284     24  0.181218               zonme
34721      1  0.194624              zumber
32715      1  0.206540              zuruck

[1949 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[25, 22, 6, 15, 12, 10, 20, 9, 11, 8, 2, 19, 16, 18, 3, 13, 5, 4, 26, 21, 23, 27, 24, 17, 7, 1, 14])

In [359]:
# Final model based on highest possible coherence score and lowest possible perplexity compromise 

# 28	0.31 symmetric	0.290924108	-9.279195784

import pickle 

lda_model_coherence_perplexity = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=28, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.31,
                                           eta='symmetric')

import pyLDAvis.gensim
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared_coherence_perplexity = pyLDAvis.gensim.prepare(lda_model_coherence_perplexity, corpus, id2word)
LDAvis_prepared_coherence_perplexity                                     

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
24     0.117556  0.012132       1        1  35.255865
26     0.169191 -0.091773       2        1   9.727740
11     0.089391  0.147391       3        1   9.498326
5      0.122525 -0.093474       4        1   6.692176
9      0.072412  0.103713       5        1   5.302658
0      0.147241 -0.145162       6        1   4.037642
27     0.089947 -0.037219       7        1   2.421645
14     0.054835  0.010574       8        1   2.176576
18     0.055136 -0.004305       9        1   2.052828
20     0.043153  0.056935      10        1   2.045929
19     0.041718  0.002182      11        1   2.032601
1      0.028519  0.055037      12        1   1.711662
17     0.020025  0.039679      13        1   1.633163
12    -0.006559  0.057144      14        1   1.536066
15     0.026205 -0.016466      15        1   1.498326
23     0.028746 -0.087042      16        1   1.477529
4      0.036157 -0.038723      17        1   1.442413
3      0.001840  0.029026      18        1   1.241227
7     -0.034699  0.091912      19        1   1.205697
21    -0.057676  0.069845      20        1   1.064624
8     -0.068318  0.068497      21        1   1.013305
6     -0.097473  0.087338      22        1   1.012135
2     -0.102299  0.020402      23        1   0.840256
25    -0.227047 -0.072881      24        1   0.768984
13    -0.090088 -0.029559      25        1   0.699698
16    -0.097350 -0.131592      26        1   0.680867
22    -0.149615 -0.041509      27        1   0.586497
10    -0.213473 -0.062101      28        1   0.343566, topic_info=         Term          Freq         Total Category  logprob  loglift
3       aller  17169.000000  17169.000000  Default  30.0000  30.0000
66      faire  28478.000000  28478.000000  Default  29.0000  29.0000
582      ouai   6200.000000   6200.000000  Default  28.0000  28.0000
176   vouloir  15727.000000  15727.000000  Default  27.0000  27.0000
122      plus  16793.000000  16793.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
122      plus     34.475358  16793.800679  Topic28  -5.0006  -0.5150
174      voir     31.713368  12162.301511  Topic28  -5.0841  -0.2758
124   pouvoir     19.585504   9364.799298  Topic28  -5.5660  -0.4964
633  toujours     17.799036   4903.872153  Topic28  -5.6617   0.0549
100       mec     16.324041   3257.082295  Topic28  -5.7482   0.3776

[2181 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
2374       1  0.030052    aaah
2374       4  0.060105    aaah
2374       5  0.120209    aaah
2374       8  0.060105    aaah
2374      16  0.721256    aaah
...      ...       ...     ...
14923     11  0.040742   zoxea
14923     12  0.122226   zoxea
14923     23  0.814837   zoxea
14924     13  0.230526  zoxeer
14924     23  0.757442  zoxeer

[9498 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[25, 27, 12, 6, 10, 1, 28, 15, 19, 21, 20, 2, 18, 13, 16, 24, 5, 4, 8, 22, 9, 7, 3, 26, 14, 17, 23, 11])

In [319]:
# testing another relatively high coherence score for comparison 
#29	0.91	0.91	0.452157515	-8.543264505

import pickle 

lda_model_test = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=29, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.91,
                                           eta=0.91)

import pyLDAvis.gensim
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared_test = pyLDAvis.gensim.prepare(lda_model_test, corpus, id2word)
LDAvis_prepared_test                                  

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
24    -0.379138 -0.005127       1        1  84.841830
9     -0.028602  0.060666       2        1   1.495394
25    -0.004794 -0.030163       3        1   0.828927
5      0.011906 -0.009120       4        1   0.718834
11     0.015483 -0.005486       5        1   0.631068
15     0.016064 -0.003492       6        1   0.561927
8      0.014498 -0.004762       7        1   0.559238
10     0.017517 -0.001734       8        1   0.551979
1      0.014279 -0.003050       9        1   0.529177
7      0.015809 -0.001872      10        1   0.528475
28     0.015906 -0.001250      11        1   0.511929
18     0.015871 -0.001420      12        1   0.504591
4      0.015936 -0.000998      13        1   0.498043
27     0.015928 -0.000767      14        1   0.497987
12     0.016578 -0.000282      15        1   0.472160
2      0.016533  0.000597      16        1   0.460260
19     0.015807  0.000250      17        1   0.458157
22     0.016697  0.000478      18        1   0.451402
20     0.016153  0.000584      19        1   0.451369
3      0.016009  0.000515      20        1   0.448100
14     0.015603  0.000425      21        1   0.447305
26     0.016101  0.000632      22        1   0.445961
16     0.016585  0.000782      23        1   0.445500
23     0.016020  0.000557      24        1   0.445440
17     0.016070  0.000708      25        1   0.445142
21     0.016012  0.000752      26        1   0.443526
6      0.016414  0.000832      27        1   0.443153
0      0.015973  0.000776      28        1   0.441667
13     0.016784  0.000972      29        1   0.441459, topic_info=          Term          Freq         Total Category  logprob  loglift
582       ouai   5179.000000   5179.000000  Default  30.0000  30.0000
1084     negro   2030.000000   2030.000000  Default  29.0000  29.0000
66       faire  25086.000000  25086.000000  Default  28.0000  28.0000
206       coup   3521.000000   3521.000000  Default  27.0000  27.0000
1808      bois    674.000000    674.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
633   toujours      0.170084   4305.361421  Topic29 -10.5630  -4.7162
582       ouai      0.167110   5179.838213  Topic29 -10.5806  -4.9188
156      temps      0.167062   4285.936226  Topic29 -10.5809  -4.7296
1099    passer      0.166699   5365.614506  Topic29 -10.5831  -4.9565
17         bon      0.166233   4339.997002  Topic29 -10.5859  -4.7472

[1741 rows x 6 columns], token_table=       Topic      Freq                Term
term                                      
27472      1  0.151064  aberration_origine
27472      7  0.302128  aberration_origine
25446      1  0.199584            abonnesc
2660       1  0.630228              abuser
2660       3  0.266635              abuser
...      ...       ...                 ...
3651       2  0.071482                zone
16284      1  0.147799               zonme
16284     23  0.295598               zonme
32715      1  0.187134              zuruck
32715     15  0.187134              zuruck

[2057 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[25, 10, 26, 6, 12, 16, 9, 11, 2, 8, 29, 19, 5, 28, 13, 3, 20, 23, 21, 4, 15, 27, 17, 24, 18, 22, 7, 1, 14])

In [19]:
# The subcorpus
import pandas as pd
import os
# Read data into papers
papers = pd.read_csv('../Experiments/metadata/subcorpus_metadata.csv')
# Print head
papers.head()

,Unnamed: 0,artistId,geniusName,doc_id,title,songId,url,text,releaseDate
0,1,16833,Aketo,AKETO1,35 Heures by Sniper,61151,https://genius.com/Sniper-35-heures-lyrics,\r\n \r\n \r\n ...,"May 19, 2003"
1,2,16833,Aketo,AKETO13,Au Pied du mur by Aketo,82390,https://genius.com/Aketo-au-pied-du-mur-lyrics,\r\n \r\n \r\n ...,000000000
2,3,1961,Akhenaton,AKHENATON5,16'30 Contre la Censure by Cercle Rouge (Ft. A...,145675,https://genius.com/Cercle-rouge-1630-contre-la...,\r\n \r\n \r\n ...,000000000
3,4,1961,Akhenaton,AKHENATON6,16 traits 16 lignes by Les Sages Poètes de la ...,3032837,https://genius.com/Les-sages-poetes-de-la-rue-...,\r\n \r\n \r\n ...,"March 24, 2017"
4,5,1961,Akhenaton,AKHENATON7,18/12/97 by Chiens de Paille (Ft. Akhenaton),4346196,https://genius.com/Chiens-de-paille-18-12-97-l...,\r\n \r\n \r\n ...,000000000


In [20]:
# Remove the columns
papers = papers.drop(columns=['artistId', 'geniusName', 'doc_id', 
                              'title', 'songId', 'url', 'releaseDate'], axis=1)
                              
# Print out the first rows of papers
papers.head()

,Unnamed: 0,text
0,1,\r\n \r\n \r\n ...
1,2,\r\n \r\n \r\n ...
2,3,\r\n \r\n \r\n ...
3,4,\r\n \r\n \r\n ...
4,5,\r\n \r\n \r\n ...


In [21]:
# Load the regular expression library
import re
# Remove punctuation
# Remove sections seprators 
papers['text'] = papers['text'].map(lambda x: re.sub(r'\[.*\]', '', x))
papers['text'] = papers['text'].map(lambda x: re.sub('couplet', '', x))
papers['text'] = papers['text'].map(lambda x: re.sub('refrain', '', x))
papers['text'] = papers['text'].map(lambda x: re.sub('intro', '', x))
papers['text'] = papers['text'].map(lambda x: re.sub('outro', '', x))
papers['text'] = papers['text'].map(lambda x: re.sub('pré-refrain', '', x))
papers['text'] = papers['text'].map(lambda x: re.sub(r'[,\.!?]', '', x))

# Convert the titles to lowercase
papers['text'] = papers['text'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['text'].head(10)

0    \r\n          \r\n            \r\n            ...
1    \r\n          \r\n            \r\n            ...
2    \r\n          \r\n            \r\n            ...
3    \r\n          \r\n            \r\n            ...
4    \r\n          \r\n            \r\n            ...
5    \r\nil nous reste plus que 24 à vivre\r\nje te...
6    ( 1 : akhenaton)\r\nj’ai connu le rap j’avais ...
7    avant de partir j'ai tant de chose à dire à mo...
8    \r\n          \r\n            \r\n            ...
9    \r\nbo banlieue ouest négro\r\ndes holsters av...
Name: text, dtype: object

In [22]:
import gensim
from gensim.utils import simple_preprocess
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = papers.text.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1][0][:30])

['suis', 'dans', 'la', 'file', 'attente', 'moi', 'et', 'mon', 'brolic', 'on', 'impatiente', 'la', 'pression', 'monte', 'passe', 'acte', 'apres', 'cette', 'cliente', 'ca', 'est', 'elle', 'barre', 'moi', 'les', 'parts', 'et', 'les', 'barres', 'je']


In [23]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [24]:
# NLTK Stop words
# import nltk
# nltk.download('stopwords')
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english') + stopwords.words('french')
# stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [25]:
import spacy
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'fr' model, keeping only tagger component (for efficiency)
nlp = spacy.load('fr_core_news_lg', disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams)
print(data_lemmatized[:1])

[['file', 'attent', 'brolic', 'impatient', 'pression', 'monte', 'passer', 'acte', 'apre', 'client', 'barre', 'barre', 'sortir', 'calibron', 'panique', 'bord', 'fermer', 'aller', 'fermer', 'porte', 'tire', 'coup', 'ok', 'personne', 'bouger', 'blesse', 'regarde', 'ouvrir', 'coffre', 'lieu', 'stresser', 'blaz', 'tuer', 'sieur', 'alors', 'entourloupe', 'sinon', 'colle', 'pruneau', 'travail', 'sale', 'aime', 'devoir', 'lever', 'autre', 'bon', 'trop', 'flegm', 'taffer', 'cdd', 'cdi', 'taffer', 'foutre', 'mazout', 'tdi', 'taffer', 'falloir', 'voir', 'pays', 'taffer', 'ok', 'taffer', 'ouai', 'taffer', 'mem', 'dos', 'esquint', 'devoir', 'dire', 'travail', 'sant', 'taffer', 'tout', 'jour', 'obliger', 'pointer', 'taffer', 'pouvoir', 'plus', 'taffer', 'merde', 'taffer', 'vie', 'merde']]


In [26]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1])
num_of_docs = len(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 10), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1)]]


In [27]:
from gensim.models import CoherenceModel

# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    perplexity = lda_model.log_perplexity(corpus)
    return [coherence_model_lda.get_coherence(), perplexity]

    
import numpy as np
import tqdm
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 30
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': [],
                 'Perplexity': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    results = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(results[0])
                    model_results['Perplexity'].append(results[1])
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results_subcorpus.csv', index=False)
    pbar.close()

1680it [5:46:13, 12.37s/it]


In [28]:
# Subcorpus Final model based on perplexity score

import pickle 

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=29, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.91,
                                           eta=0.01)

import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared         

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.121634  0.007816       1        1  16.638812
3     -0.121169 -0.051533       2        1  10.274036
2     -0.134899 -0.011239       3        1   6.151365
7     -0.067034 -0.080332       4        1   4.879635
6     -0.106992 -0.014338       5        1   4.559712
15    -0.083130 -0.002571       6        1   4.389850
23    -0.096443 -0.019940       7        1   3.841986
28    -0.086420 -0.018720       8        1   3.789798
20    -0.086294  0.020191       9        1   3.725300
18    -0.081227 -0.000772      10        1   3.686643
9     -0.104084  0.050425      11        1   3.470796
12    -0.105792  0.081322      12        1   3.340333
27    -0.091395  0.047572      13        1   3.051860
11    -0.050200  0.048980      14        1   2.962791
16     0.007307  0.148340      15        1   2.723440
14     0.007501  0.020519      16        1   2.511947
0     -0.001916 -0.030015      17        1   2.398074
13     0.044701 -0.076970      18        1   2.249671
25     0.055643  0.132368      19        1   2.182974
26     0.041177 -0.067050      20        1   1.975113
8      0.076685 -0.058090      21        1   1.765275
4      0.062704 -0.086728      22        1   1.589170
1      0.209982  0.266523      23        1   1.425188
24     0.132115  0.153396      24        1   1.368334
22     0.116007 -0.076557      25        1   1.256800
21     0.160005 -0.091808      26        1   1.134546
17     0.122025 -0.105954      27        1   1.081829
10     0.172420 -0.079591      28        1   0.887567
19     0.130356 -0.105245      29        1   0.687153, topic_info=        Term         Freq        Total Category  logprob  loglift
75     faire  1664.000000  1664.000000  Default  30.0000  30.0000
2      aller  1084.000000  1084.000000  Default  29.0000  29.0000
1031     oui   204.000000   204.000000  Default  28.0000  28.0000
9894  ah_oui   150.000000   150.000000  Default  27.0000  27.0000
132   savoir   623.000000   623.000000  Default  26.0000  26.0000
...      ...          ...          ...      ...      ...      ...
131      rap     1.998948   349.386264  Topic29  -5.8252  -0.1832
284   putain     1.033354   179.921129  Topic29  -6.4850  -0.1793
268    finir     1.059921   190.087192  Topic29  -6.4596  -0.2089
97     temps     1.356225   321.935160  Topic29  -6.2131  -0.4893
287    reste     1.224103   271.515062  Topic29  -6.3156  -0.4214

[1986 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
877        1  0.135253  Monsieur
877       20  0.067627  Monsieur
877       22  0.541012  Monsieur
877       23  0.270506  Monsieur
15205      2  0.982984   abjects
...      ...       ...       ...
11372      2  0.956842       zen
2319       3  0.212966      zero
2319      14  0.745383      zero
728        4  0.988498     zesau
3283      12  1.014583      ziqu

[5068 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 3, 8, 7, 16, 24, 29, 21, 19, 10, 13, 28, 12, 17, 15, 1, 14, 26, 27, 9, 5, 2, 25, 23, 22, 18, 11, 20])

In [29]:
# Subcorpus Final model based on coherence score

import pickle 

lda_model_coherence = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=19, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.91,
                                           eta=0.91)

import pyLDAvis.gensim
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared_coherence = pyLDAvis.gensim.prepare(lda_model_coherence, corpus, id2word)
LDAvis_prepared_coherence                                          

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.206463 -0.001895       1        1  74.915152
3     -0.053911  0.040386       2        1   3.898834
7     -0.026468 -0.056742       3        1   2.657452
16     0.014124  0.004535       4        1   1.590193
15     0.000214  0.000263       5        1   1.488386
13     0.018761  0.001516       6        1   1.408117
8      0.012159  0.003744       7        1   1.401597
2      0.015348  0.002103       8        1   1.304450
1      0.026336  0.001008       9        1   1.247269
4      0.013222  0.002082      10        1   1.179542
6      0.012917  0.000504      11        1   1.165005
12     0.020930  0.000605      12        1   1.019276
18     0.021372  0.000108      13        1   1.018455
9      0.021025  0.001101      14        1   1.009529
0      0.022272  0.000167      15        1   0.971727
14     0.022062  0.000125      16        1   0.965644
11     0.021655  0.000256      17        1   0.955506
10     0.022462  0.000113      18        1   0.902334
17     0.021982  0.000021      19        1   0.901533, topic_info=        Term         Freq        Total Category  logprob  loglift
75     faire  1088.000000  1088.000000  Default  30.0000  30.0000
140  vouloir   633.000000   633.000000  Default  29.0000  29.0000
2      aller   776.000000   776.000000  Default  28.0000  28.0000
50   pouvoir   344.000000   344.000000  Default  27.0000  27.0000
480    aussi    96.000000    96.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
47      plus     0.082160   785.936290  Topic19  -9.2884  -4.4571
68      voir     0.079766   534.233368  Topic19  -9.3180  -4.1007
140  vouloir     0.077906   633.539092  Topic19  -9.3416  -4.2947
22      dire     0.075700   597.004621  Topic19  -9.3703  -4.2641
132   savoir     0.072885   439.974519  Topic19  -9.4082  -3.9968

[1069 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
15205      1  0.282014   abjects
15205      2  0.564029   abjects
4467       1  0.081907   accepte
4467      12  0.819072   accepte
8940       1  0.453055  acclamer
...      ...       ...       ...
2525       8  0.014458        yo
728        1  0.211195     zesau
728        3  0.422389     zesau
3283       1  0.249043      ziqu
3283       2  0.498085      ziqu

[1431 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 8, 17, 16, 14, 9, 3, 2, 5, 7, 13, 19, 10, 1, 15, 12, 11, 18])

In [30]:
# Final model based on compromise

import pickle 

lda_model_coherence = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=16, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.61,
                                           eta=0.01)

import pyLDAvis.gensim
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared_coherence = pyLDAvis.gensim.prepare(lda_model_coherence, corpus, id2word)
LDAvis_prepared_coherence                                          

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.069012  0.032717       1        1  22.607516
3      0.079819  0.031995       2        1  13.451601
2      0.082281  0.041428       3        1   8.956660
7      0.056446  0.060071       4        1   7.351644
9      0.077519  0.015540       5        1   6.968522
15     0.031997  0.003648       6        1   5.949891
11     0.046231  0.007141       7        1   5.718808
6      0.064026  0.011155       8        1   5.561552
12     0.039880 -0.012395       9        1   4.644767
4      0.043780  0.073033      10        1   4.547730
8     -0.034321  0.005978      11        1   3.157146
14    -0.011203 -0.046261      12        1   2.966341
13    -0.050704 -0.344851      13        1   2.902505
0     -0.013211  0.048951      14        1   2.250121
10    -0.059041 -0.009555      15        1   1.563048
1     -0.422509  0.081408      16        1   1.402148, topic_info=        Term         Freq        Total Category  logprob  loglift
75     faire  1603.000000  1603.000000  Default  30.0000  30.0000
47      plus  1013.000000  1013.000000  Default  29.0000  29.0000
50   pouvoir   517.000000   517.000000  Default  28.0000  28.0000
597     sens   152.000000   152.000000  Default  27.0000  27.0000
140  vouloir   843.000000   843.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
549   maison     6.644858   100.444416  Topic16  -5.3371   1.5514
475     vrai     6.829004   194.174271  Topic16  -5.3098   0.9196
225   parler     6.799477   206.518612  Topic16  -5.3141   0.8536
87      pied     6.525747    84.368450  Topic16  -5.3552   1.7077
132   savoir     6.280985   556.741875  Topic16  -5.3935  -0.2174

[1239 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
15205      2  0.973993  abjects
4467       9  0.989772  accepte
7687      12  0.936181   acheve
2832       1  0.189441   action
2832       3  0.027063   action
...      ...       ...      ...
13240     15  1.010387      zed
6123       2  1.005053     zele
11372      2  1.017354      zen
728        4  0.991769    zesau
3283       9  1.044769     ziqu

[3089 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 4, 3, 8, 10, 16, 12, 7, 13, 5, 9, 15, 14, 1, 11, 2])